<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-393 Máquinas de Aprendizaje II-2018 </h1>

<H3 align='center'> Fernanda Urrea, ROL: 201551522-0 </H3>
<H3 align='center'> Matías Gómez, ROL: 201460501-3 </H3>

<hr style="height:2px;border:none"/>

## 4. Entendimiento de imágenes de personas

El problema de inferir ciertas características de una persona a través de una foto de ella puede resultar bastante difícil incluso para nosotros, como por ejemplo de qué país es, la emoción que expresa, la edad que tiene, o el género. La automatización de este proceso para que máquinas logren identificar ciertas características de una persona puede ser algo crucial para el futuro desarrollo de Inteligencia Artificial.


<img src="https://i.imgur.com/6B072GE.jpg" width="60%" height="20%" />


En esta actividad trabajaremos con unos datos (imágenes) con la tarea de predecir la **edad** (*target value*) de la persona en la imagen. Los datos con corresponden a 3640 imágenes de Flickr de rostros de personas, pero para simplificar el manejo y cómputo, se trabajará con representaciones de características extraídas (descriptores). Para ésto necesitará descargar los datos del siguiente __[link](http://chenlab.ece.cornell.edu/people/Andy/ImagesOfGroups.html)__ en el extracto de *ageGenderClassification* o a través de la consola Unix.
```
wget http://chenlab.ece.cornell.edu/projects/ImagesOfGroups/ageGenderClassification.zip
```

Se trabajará con archivos *.mat* que pueden ser cargados de la siguiente manera:

```python
import scipy.io as sio
sio.loadmat("file.mat")
```

Para descripción sobre las columnas y metadatos del archivo descargado favor dirigirse al archivo readme a través del siguiente __[link](http://chenlab.ece.cornell.edu/projects/ImagesOfGroups/README.txt)__ o a través de la consola Unix:
```
wget http://chenlab.ece.cornell.edu/projects/ImagesOfGroups/README.txt
```
En el apartado "*MATLAB DATA*".

#### a) Cargue los datos dos dataset de entrenamiento y de pruebas ¿Cuántos datos hay en cada conjunto?

In [1]:
import scipy.io as sio
mat_train = sio.loadmat("./AgeGenderClassification/eventrain.mat")
mat_test = sio.loadmat("./AgeGenderClassification/eventest.mat")
data_train= mat_train["trcoll"][0][0][0]
data_test= mat_test["tecoll"][0][0][0]

In [2]:
print(data_train.shape)
print(data_train[0])
print(data_test.shape)
print(data_test[0])

(3500, 14)
[42.         34.          3.          0.98386991  6.         -0.36363636
  0.1798535  -0.          1.04286927 -3.49598221  0.11850787  1.04375052
  1.          5.        ]
(1050, 14)
[87.         35.          1.          1.11803399  4.15908644  0.26832816
 -0.         -0.1798535   1.0615004  16.11449821 -2.21822498  1.09343236
  1.          5.        ]


El dataset de entranamiento contiene 3.500 ejemplos, y cada ejemplo es una 14-tupla de números.  
El dataset de pruebas contiene 1.050 ejemplos, y cada ejemplo es una 14-tupla de números.  

#### b) Elija cuál representación utilizará para trabajar los datos y entregárselos como *input* al modelo de aprendizaje a utilizar, recuerde que puede utilizar una combinación de éstos si lo desea. Además extraiga las salidas/*output* del problema, en este caso, como ya se comentó, la edad. Describa los datos utilizados y la cantidad de datos por rango de edad.

In [4]:
#genFeat = data[0]  #it can be used as representation: contextual features
#ageClass = data[1] #target
#ffcoefs = data[3]   #it can be used as representation: fisherface space
#faceGist = data[4]  #it can be used as representation: face

#Se utilizaran los 3 atributos presentados antes
#Se crean los conjuntos de prueba inputs y outputs
import numpy as np;

Y_train=[];
X_train=[];
Y_test=[];
X_test=[];

for i in range(3500):
    Y_train.append(int(data_train[i][1]));
    l=[];
    l.append(data_train[i][0]);
    l.append(data_train[i][2]);
    l.append(data_train[i][3]);
    X_train.append(l);
    if i<1050:
        Y_test.append(int(data_test[i][1]));
        l=[];
        l.append(data_test[i][0]);
        l.append(data_test[i][2]);
        l.append(data_test[i][3]);
        X_test.append(l);
        
Y_train=np.asarray(Y_train);
X_train=np.asarray(X_train);
Y_test=np.asarray(Y_test);
X_test=np.asarray(X_test);

print(Y_train.shape,Y_train[0]);
print(X_train.shape,X_train[0]);
print(Y_test.shape,Y_test[0]);
print(X_test.shape,X_test[0]);

(3500,) 34
(3500, 3) [42.          3.          0.98386991]
(1050,) 35
(1050, 3) [87.          1.          1.11803399]


Se crean los conjuntos de entrenamiento y pruebas.  
X_train tiene 3.500 ejemplos con 3 atributos cada uno.  
Y_train contiene las 3.500 edades que corresponden a los 3.500 ejemplos.  
X_test tiene 1.050 ejemplos con 3 atributos cada uno.  
Y_test contiene las 1.050 edades que corresponden a los 1.050 ejemplos.

In [5]:
cantidad_edades_train = np.bincount(Y_train);
numeros_edades_train = np.nonzero(cantidad_edades_train)[0];
#Arreglo de listas de dos elementos donde el primero es la edad y el segundo la cantidad
arreglo_train=np.vstack((numeros_edades_train,cantidad_edades_train[numeros_edades_train])).T; 

cantidad_edades_test = np.bincount(Y_test);
numeros_edades_test = np.nonzero(cantidad_edades_test)[0];
#Arreglo de listas de dos elementos donde el primero es la edad y el segundo la cantidad
arreglo_test=np.vstack((numeros_edades_test,cantidad_edades_test[numeros_edades_test])).T;

print(arreglo_train,"\n\n",arreglo_test)

[[ 5  1]
 [ 6  6]
 [ 7  3]
 [ 8  6]
 [ 9 12]
 [10 19]
 [11 15]
 [12 20]
 [13 20]
 [14 33]
 [15 38]
 [16 41]
 [17 43]
 [18 62]
 [19 54]
 [20 58]
 [21 76]
 [22 62]
 [23 64]
 [24 82]
 [25 78]
 [26 69]
 [27 74]
 [28 53]
 [29 89]
 [30 67]
 [31 67]
 [32 78]
 [33 59]
 [34 68]
 [35 75]
 [36 65]
 [37 71]
 [38 75]
 [39 70]
 [40 78]
 [41 71]
 [42 78]
 [43 49]
 [44 62]
 [45 64]
 [46 75]
 [47 58]
 [48 57]
 [49 71]
 [50 71]
 [51 57]
 [52 65]
 [53 59]
 [54 62]
 [55 58]
 [56 49]
 [57 49]
 [58 42]
 [59 48]
 [60 47]
 [61 34]
 [62 33]
 [63 37]
 [64 32]
 [65 34]
 [66 24]
 [67 33]
 [68 31]
 [69 21]
 [70 27]
 [71 23]
 [72 12]
 [73 18]
 [74 13]
 [75 15]
 [76 10]
 [77  8]
 [78 10]
 [79  6]
 [80  7]
 [81  5]
 [82  3]
 [83  1]
 [84  5]
 [85  1]
 [86  3]
 [87  4]
 [88  3]
 [89  1]
 [90  1]
 [91  1]
 [97  1]] 

 [[ 6  1]
 [ 7  3]
 [ 8  2]
 [ 9  5]
 [10  9]
 [11  7]
 [12 11]
 [13  4]
 [14  5]
 [15  8]
 [16 15]
 [17 12]
 [18 17]
 [19 20]
 [20 17]
 [21 14]
 [22 13]
 [23 16]
 [24 15]
 [25 16]
 [26 19]
 [27 21]
 [28 2

Se crearon arreglos, en los que se detallan edades exactas y cantidad de ejemplos para cada edad en los conjuntos de entrenamiento y prueba.  
A continuación, se convertirán las edades en los rangos que se detallan en el documento README.txt del enunciado, es decir:  
<table>
    <tr>
        <td>
\begin{eqnarray}
&{\textbf{Etiqueta}}\\ & 1 \\ & 5\\ & 10\\ & 16\\ & 28\\ & 51\\ & 75\\
\end{eqnarray}
        </td>
        <td>
\begin{eqnarray}
&{\textbf{Rango etario}}\\& 0-2 \\ & 3-7\\ & 8-12\\ & 13-19\\ & 20-36\\ & 37-65\\ & 66+\\
\end{eqnarray}
        </td>


    </tr>
</table>

In [6]:
for i in range(3500):
    if Y_train[i]<3:
        Y_train[i]=1;
    elif Y_train[i]<8:
        Y_train[i]=5;
    elif Y_train[i]<13:
        Y_train[i]=10;
    elif Y_train[i]<20:
        Y_train[i]=16;
    elif Y_train[i]<37:
        Y_train[i]=28;
    elif Y_train[i]<66:
        Y_train[i]=51;
    else:
        Y_train[i]=75;
    if i<1050:
        if Y_test[i]<3:
            Y_test[i]=1;
        elif Y_test[i]<8:
            Y_test[i]=5;
        elif Y_test[i]<13:
            Y_test[i]=10;
        elif Y_test[i]<20:
            Y_test[i]=16;
        elif Y_test[i]<37:
            Y_test[i]=28;
        elif Y_test[i]<66:
            Y_test[i]=51;
        else:
            Y_test[i]=75;
        

In [7]:
cantidad_edades_trainNew = np.bincount(Y_train);
numeros_edades_trainNew = np.nonzero(cantidad_edades_trainNew)[0];
#Arreglo de listas de dos elementos donde el primero es la edad y el segundo la cantidad
arreglo_trainNew=np.vstack((numeros_edades_trainNew,cantidad_edades_trainNew[numeros_edades_trainNew])).T; 

cantidad_edades_testNew = np.bincount(Y_test);
numeros_edades_testNew = np.nonzero(cantidad_edades_testNew)[0];
#Arreglo de listas de dos elementos donde el primero es la edad y el segundo la cantidad
arreglo_testNew=np.vstack((numeros_edades_testNew,cantidad_edades_testNew[numeros_edades_testNew])).T;

print(arreglo_trainNew,"\n\n",arreglo_testNew)

[[   5   10]
 [  10   72]
 [  16  291]
 [  28 1184]
 [  51 1656]
 [  75  287]] 

 [[  5   4]
 [ 10  34]
 [ 16  81]
 [ 28 329]
 [ 51 518]
 [ 75  84]]


Así, los outputs del conjunto de $\textbf{entrenamiento}$ quedan:
<table>
    <tr>
        <td>
\begin{eqnarray}
&{\textbf{Etiqueta}}\\  & 5\\ & 10\\ & 16\\ & 28\\ & 51\\ & 75\\
\end{eqnarray}
        </td>
        <td>
\begin{eqnarray}
&{\textbf{Cantidad}}\\& 10 \\ & 72\\ & 291\\ & 1184\\ & 1656\\ & 287\\
\end{eqnarray}
        </td>
    </tr>
</table>
  
Y los outputs del conjunto de $\textbf{pruebas}$ quedan:
<table>
    <tr>
        <td>
\begin{eqnarray}
&{\textbf{Etiqueta}}\\  & 5\\ & 10\\ & 16\\ & 28\\ & 51\\ & 75\\
\end{eqnarray}
        </td>
        <td>
\begin{eqnarray}
&{\textbf{Cantidad}}\\& 4 \\ & 34\\ & 81\\ & 329\\ & 518\\ & 84\\
\end{eqnarray}
        </td>
    </tr>
</table>

#### c) Defina y entrene modelos de aprendizaje para la inferencia de la edad de la persona a través de la representación escogida, *se espera que experimente con distintas configuraciones, modelos e hiper-parámetros* . Intente llegar a un *MSE* menor a 100 sobre la edad de las personas en el conjunto de pruebas. Recuerde que **NO** puede seleccionar modelos a través del conjunto de pruebas. Visualice sus resultados si estima conveniente.

*Nota: Puede notar que la cantidad de edades presentes en el problema son pocas (1,  5, 10, 16, 28, 51 o 75 años), por lo que puede tratar al problema así como de regresión o clasificación (considerando cada edad como una clase)*

In [56]:
from sklearn.metrics import mean_squared_error as MSE

Se considerará el problema como uno de clasificación asignando cada edad como una clase.

In [8]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_test_scaled = std.transform(X_test)

Se procede a crear una representación de las clases etiquetándolas de $0$ a $6$.  

In [10]:
Y_train_codif=[];
Y_test_codif=[];
for i in range(3500):
    if Y_train[i]==1:
        Y_train_codif.append(0)
    if Y_train[i]==5:
        Y_train_codif.append(1)        
    if Y_train[i]==10:
        Y_train_codif.append(2)  
    if Y_train[i]==16:
        Y_train_codif.append(3)  
    if Y_train[i]==28:
        Y_train_codif.append(4)  
    if Y_train[i]==51:
        Y_train_codif.append(5)  
    if Y_train[i]==75:
        Y_train_codif.append(6)  
    if i<1050:
        if Y_test[i]==1:
            Y_test_codif.append(0)
        if Y_test[i]==5:
            Y_test_codif.append(1)        
        if Y_test[i]==10:
            Y_test_codif.append(2)  
        if Y_test[i]==16:
            Y_test_codif.append(3)  
        if Y_test[i]==28:
            Y_test_codif.append(4)  
        if Y_test[i]==51:
            Y_test_codif.append(5)  
        if Y_test[i]==75:
            Y_test_codif.append(6) 
Y_train_codif=np.asarray(Y_train_codif);
Y_test_codif=np.asarray(Y_test_codif);
print(Y_train_codif.shape,Y_train_codif)

(3500,) [4 2 4 ... 5 6 5]


En base a la representaicón anterior, se construye con ayuda de keras una representación $\textit{one hot vector}$. Luego se presentan algunos ejemplos.  

In [14]:
import keras
y_onehot_train = keras.utils.to_categorical(Y_train_codif,num_classes=7)
y_onehot_test = keras.utils.to_categorical(Y_test_codif,num_classes=7)

In [37]:
print(y_onehot_train[0], Y_train[0]);
print(y_onehot_train[20],  Y_train[20]);
print(y_onehot_train[40],  Y_train[40]);
print(y_onehot_train[60],  Y_train[60]);
print(y_onehot_train[100],  Y_train[100]);
print(y_onehot_test[0], Y_test[0]);
print(y_onehot_test[20], Y_test[20]);
print(y_onehot_test[40], Y_test[40]);
print(y_onehot_test[60], Y_test[60]);
print(y_onehot_test[100], Y_test[100]);

[0. 0. 0. 0. 1. 0. 0.] 28
[0. 0. 0. 0. 0. 1. 0.] 51
[0. 0. 0. 0. 1. 0. 0.] 28
[0. 0. 0. 0. 1. 0. 0.] 28
[0. 0. 0. 0. 1. 0. 0.] 28
[0. 0. 0. 0. 1. 0. 0.] 28
[0. 0. 0. 1. 0. 0. 0.] 16
[0. 0. 0. 0. 0. 1. 0.] 51
[0. 0. 0. 0. 0. 1. 0.] 51
[0. 0. 0. 0. 1. 0. 0.] 28


Como el problema es de clasificación, utilizaremos una red neuronal de modelo $\textit{sequential}$, pero a diferencia de la pregunta 2, la función de activación será $\textit{softmax}$ y la función de pérdida $\textit{categorical_crossentropy}$.  

In [66]:
import time
start_time =time.time()
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(y_onehot_train.shape[1], activation="softmax"))
model.compile(optimizer="Adam", loss="categorical_crossentropy")
model.fit(X_train_scaled, y_onehot_train, epochs=40, batch_size=128, verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/40
3500/3500 [==============================] - 1s 232us/step - loss: 1.5137
Epoch 2/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.2355
Epoch 3/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.2031
Epoch 4/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1915
Epoch 5/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1861
Epoch 6/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1827
Epoch 7/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1808
Epoch 8/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1801
Epoch 9/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1780
Epoch 10/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1757
Epoch 11/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1749
Epoch 12/40
3500/3500 [==============================] - 0s 27us/step - l

Se procede a trabajar los output de la red neuronal para volverlos al formato original, con el fin de poder calcular el error.

In [67]:
y_pred_train_nn=model.predict(X_train_scaled);
y_pred_test_nn=model.predict(X_test_scaled);

In [68]:
#print(len(y_pred_train_nn),len(y_pred_test_nn))
def f(x):
    if x==0:
        return 1;
    elif x==1:
        return 5;
    elif x==2:
        return 10;
    elif x==3:
        return 16;
    elif x==4:
        return 28;
    elif x==5:
        return 51;
    elif x==6:
        return 75;


y_pred_train_nn_etiquetas=[];
y_pred_test_nn_etiquetas=[];
#print(len(y_pred_train_nn[0]))
for i in range(3500):
    posMax=6;
    valMax=y_pred_train_nn[i][6]
    for j in range(6):
        if y_pred_train_nn[i][j] > valMax:
            valMax=y_pred_train_nn[i][j];
            posMax=j;
            
    y_pred_train_nn_etiquetas.append(f(posMax));
    if i < 1050:
        posMax=6;
        valMax=y_pred_test_nn[i][6]
        for j in range(6):
            if y_pred_test_nn[i][j] > valMax:
                valMax=y_pred_test_nn[i][j];
                posMax=j;
        y_pred_test_nn_etiquetas.append(f(posMax));   
        
y_pred_train_nn_etiquetas=np.asarray(y_pred_train_nn_etiquetas);
y_pred_test_nn_etiquetas=np.asarray(y_pred_test_nn_etiquetas);

print(y_pred_train_nn_etiquetas.shape,y_pred_train_nn_etiquetas,Y_train);
print(y_pred_test_nn_etiquetas.shape,y_pred_test_nn_etiquetas, Y_test);

(3500,) [28 51 51 ... 51 51 51] [28 10 28 ... 51 75 51]
(1050,) [28 51 51 ... 51 51 51] [28 51 51 ... 51 51 10]


In [69]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_nn_etiquetas))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_nn_etiquetas))

ERROR MSE de training:  347.50142857142856
ERROR MSE de test:  359.18857142857144


Ahora probaremos con la misma red neuronal cambiando el optimizador, de Adam a el usado en la pregunta 2, SGD.

In [72]:
import time
start_time =time.time()
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(y_onehot_train.shape[1], activation="softmax"))
model.compile(optimizer=SGD(lr=0.1), loss="categorical_crossentropy")
model.fit(X_train_scaled, y_onehot_train, epochs=40, batch_size=128, verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/40
3500/3500 [==============================] - 1s 221us/step - loss: 1.4862
Epoch 2/40
3500/3500 [==============================] - 0s 41us/step - loss: 1.2542
Epoch 3/40
3500/3500 [==============================] - 0s 32us/step - loss: 1.2223
Epoch 4/40
3500/3500 [==============================] - 0s 29us/step - loss: 1.2093
Epoch 5/40
3500/3500 [==============================] - 0s 33us/step - loss: 1.2016
Epoch 6/40
3500/3500 [==============================] - 0s 30us/step - loss: 1.1960
Epoch 7/40
3500/3500 [==============================] - 0s 34us/step - loss: 1.1915
Epoch 8/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.1876
Epoch 9/40
3500/3500 [==============================] - 0s 27us/step - loss: 1.1860
Epoch 10/40
3500/3500 [==============================] - 0s 27us/step - loss: 1.1832
Epoch 11/40
3500/3500 [==============================] - 0s 30us/step - loss: 1.1817
Epoch 12/40
3500/3500 [==============================] - 0s 30us/step - l

Se procede a trabajar los output de la red neuronal para volverlos al formato original, con el fin de poder calcular el error.

In [73]:
y_pred_train_nn=model.predict(X_train_scaled);
y_pred_test_nn=model.predict(X_test_scaled);

In [74]:
def f(x):
    if x==0:
        return 1;
    elif x==1:
        return 5;
    elif x==2:
        return 10;
    elif x==3:
        return 16;
    elif x==4:
        return 28;
    elif x==5:
        return 51;
    elif x==6:
        return 75;


y_pred_train_nn_etiquetas=[];
y_pred_test_nn_etiquetas=[];
#print(len(y_pred_train_nn[0]))
for i in range(3500):
    posMax=6;
    valMax=y_pred_train_nn[i][6]
    for j in range(6):
        if y_pred_train_nn[i][j] > valMax:
            valMax=y_pred_train_nn[i][j];
            posMax=j;
            
    y_pred_train_nn_etiquetas.append(f(posMax));
    if i < 1050:
        posMax=6;
        valMax=y_pred_test_nn[i][6]
        for j in range(6):
            if y_pred_test_nn[i][j] > valMax:
                valMax=y_pred_test_nn[i][j];
                posMax=j;
        y_pred_test_nn_etiquetas.append(f(posMax));   
        
y_pred_train_nn_etiquetas=np.asarray(y_pred_train_nn_etiquetas);
y_pred_test_nn_etiquetas=np.asarray(y_pred_test_nn_etiquetas);

print(y_pred_train_nn_etiquetas.shape,y_pred_train_nn_etiquetas,Y_train);
print(y_pred_test_nn_etiquetas.shape,y_pred_test_nn_etiquetas, Y_test);

(3500,) [51 51 51 ... 51 51 51] [28 10 28 ... 51 75 51]
(1050,) [28 51 51 ... 51 51 51] [28 51 51 ... 51 51 10]


In [75]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_nn_etiquetas))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_nn_etiquetas))

ERROR MSE de training:  358.95542857142857
ERROR MSE de test:  364.7085714285714


Se variarán los parámetros de fit, en particular la cantidad de veces que se entrena, epochs de $40$ a $15$

In [96]:
import time
start_time =time.time()
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(y_onehot_train.shape[1], activation="softmax"))
model.compile(optimizer="Adam", loss="categorical_crossentropy")
model.fit(X_train_scaled, y_onehot_train, epochs=15, batch_size=128, verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/15
3500/3500 [==============================] - 1s 312us/step - loss: 1.5524
Epoch 2/15
3500/3500 [==============================] - 0s 45us/step - loss: 1.2395
Epoch 3/15
3500/3500 [==============================] - 0s 45us/step - loss: 1.2069
Epoch 4/15
3500/3500 [==============================] - ETA: 0s - loss: 1.204 - 0s 45us/step - loss: 1.1924
Epoch 5/15
3500/3500 [==============================] - 0s 42us/step - loss: 1.1885
Epoch 6/15
3500/3500 [==============================] - 0s 45us/step - loss: 1.1865
Epoch 7/15
3500/3500 [==============================] - 0s 40us/step - loss: 1.1804
Epoch 8/15
3500/3500 [==============================] - 0s 45us/step - loss: 1.1808
Epoch 9/15
3500/3500 [==============================] - 0s 40us/step - loss: 1.1776
Epoch 10/15
3500/3500 [==============================] - 0s 36us/step - loss: 1.1797
Epoch 11/15
3500/3500 [==============================] - 0s 40us/step - loss: 1.1746
Epoch 12/15
3500/3500 [==========================

In [97]:
y_pred_train_nn=model.predict(X_train_scaled);
y_pred_test_nn=model.predict(X_test_scaled);

In [98]:
def f(x):
    if x==0:
        return 1;
    elif x==1:
        return 5;
    elif x==2:
        return 10;
    elif x==3:
        return 16;
    elif x==4:
        return 28;
    elif x==5:
        return 51;
    elif x==6:
        return 75;


y_pred_train_nn_etiquetas=[];
y_pred_test_nn_etiquetas=[];
#print(len(y_pred_train_nn[0]))
for i in range(3500):
    posMax=6;
    valMax=y_pred_train_nn[i][6]
    for j in range(6):
        if y_pred_train_nn[i][j] > valMax:
            valMax=y_pred_train_nn[i][j];
            posMax=j;
            
    y_pred_train_nn_etiquetas.append(f(posMax));
    if i < 1050:
        posMax=6;
        valMax=y_pred_test_nn[i][6]
        for j in range(6):
            if y_pred_test_nn[i][j] > valMax:
                valMax=y_pred_test_nn[i][j];
                posMax=j;
        y_pred_test_nn_etiquetas.append(f(posMax));   
        
y_pred_train_nn_etiquetas=np.asarray(y_pred_train_nn_etiquetas);
y_pred_test_nn_etiquetas=np.asarray(y_pred_test_nn_etiquetas);

print(y_pred_train_nn_etiquetas.shape,y_pred_train_nn_etiquetas,Y_train);
print(y_pred_test_nn_etiquetas.shape,y_pred_test_nn_etiquetas, Y_test);

(3500,) [28 51 51 ... 51 51 51] [28 10 28 ... 51 75 51]
(1050,) [28 51 51 ... 51 51 51] [28 51 51 ... 51 51 10]


In [99]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_nn_etiquetas))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_nn_etiquetas))

ERROR MSE de training:  341.114
ERROR MSE de test:  348.1485714285714


Se varian los parámetros de $Dense$ referentes a la cantidad de neuronas ocultas, primero con valores prqueños, luego con valores más grandes.

In [108]:
import time
start_time =time.time()
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(y_onehot_train.shape[1], activation="softmax"))
model.compile(optimizer="Adam", loss="categorical_crossentropy")
model.fit(X_train_scaled, y_onehot_train, epochs=40, batch_size=128, verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/40
3500/3500 [==============================] - 1s 342us/step - loss: 1.8693
Epoch 2/40
3500/3500 [==============================] - 0s 36us/step - loss: 1.7346
Epoch 3/40
3500/3500 [==============================] - 0s 27us/step - loss: 1.6096
Epoch 4/40
3500/3500 [==============================] - 0s 27us/step - loss: 1.4970
Epoch 5/40
3500/3500 [==============================] - 0s 27us/step - loss: 1.4065
Epoch 6/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.3442
Epoch 7/40
3500/3500 [==============================] - 0s 27us/step - loss: 1.3031
Epoch 8/40
3500/3500 [==============================] - 0s 22us/step - loss: 1.2764
Epoch 9/40
3500/3500 [==============================] - 0s 30us/step - loss: 1.2576
Epoch 10/40
3500/3500 [==============================] - 0s 22us/step - loss: 1.2440
Epoch 11/40
3500/3500 [==============================] - 0s 31us/step - loss: 1.2335
Epoch 12/40
3500/3500 [==============================] - 0s 27us/step - l

In [109]:
y_pred_train_nn=model.predict(X_train_scaled);
y_pred_test_nn=model.predict(X_test_scaled);

In [110]:
def f(x):
    if x==0:
        return 1;
    elif x==1:
        return 5;
    elif x==2:
        return 10;
    elif x==3:
        return 16;
    elif x==4:
        return 28;
    elif x==5:
        return 51;
    elif x==6:
        return 75;


y_pred_train_nn_etiquetas=[];
y_pred_test_nn_etiquetas=[];
#print(len(y_pred_train_nn[0]))
for i in range(3500):
    posMax=6;
    valMax=y_pred_train_nn[i][6]
    for j in range(6):
        if y_pred_train_nn[i][j] > valMax:
            valMax=y_pred_train_nn[i][j];
            posMax=j;
            
    y_pred_train_nn_etiquetas.append(f(posMax));
    if i < 1050:
        posMax=6;
        valMax=y_pred_test_nn[i][6]
        for j in range(6):
            if y_pred_test_nn[i][j] > valMax:
                valMax=y_pred_test_nn[i][j];
                posMax=j;
        y_pred_test_nn_etiquetas.append(f(posMax));   
        
y_pred_train_nn_etiquetas=np.asarray(y_pred_train_nn_etiquetas);
y_pred_test_nn_etiquetas=np.asarray(y_pred_test_nn_etiquetas);

print(y_pred_train_nn_etiquetas.shape,y_pred_train_nn_etiquetas,Y_train);
print(y_pred_test_nn_etiquetas.shape,y_pred_test_nn_etiquetas, Y_test);

(3500,) [28 51 51 ... 51 51 51] [28 10 28 ... 51 75 51]
(1050,) [51 51 51 ... 51 51 51] [28 51 51 ... 51 51 10]


In [111]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_nn_etiquetas))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_nn_etiquetas))

ERROR MSE de training:  349.93285714285713
ERROR MSE de test:  349.4847619047619


In [104]:
import time
start_time =time.time()
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(1000, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(y_onehot_train.shape[1], activation="softmax"))
model.compile(optimizer="Adam", loss="categorical_crossentropy")
model.fit(X_train_scaled, y_onehot_train, epochs=40, batch_size=128, verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/40
3500/3500 [==============================] - 3s 749us/step - loss: 1.3277
Epoch 2/40
3500/3500 [==============================] - 2s 436us/step - loss: 1.2051 0s - los
Epoch 3/40
3500/3500 [==============================] - 2s 499us/step - loss: 1.1907
Epoch 4/40
3500/3500 [==============================] - 2s 519us/step - loss: 1.1900
Epoch 5/40
3500/3500 [==============================] - 2s 463us/step - loss: 1.1989
Epoch 6/40
3500/3500 [==============================] - 2s 452us/step - loss: 1.1889
Epoch 7/40
3500/3500 [==============================] - 2s 506us/step - loss: 1.1805
Epoch 8/40
3500/3500 [==============================] - 1s 423us/step - loss: 1.1769
Epoch 9/40
3500/3500 [==============================] - 1s 394us/step - loss: 1.1790
Epoch 10/40
3500/3500 [==============================] - 1s 371us/step - loss: 1.1753
Epoch 11/40
3500/3500 [==============================] - 1s 366us/step - loss: 1.1775
Epoch 12/40
3500/3500 [==============================]

In [105]:
y_pred_train_nn=model.predict(X_train_scaled);
y_pred_test_nn=model.predict(X_test_scaled);

In [106]:
def f(x):
    if x==0:
        return 1;
    elif x==1:
        return 5;
    elif x==2:
        return 10;
    elif x==3:
        return 16;
    elif x==4:
        return 28;
    elif x==5:
        return 51;
    elif x==6:
        return 75;


y_pred_train_nn_etiquetas=[];
y_pred_test_nn_etiquetas=[];
#print(len(y_pred_train_nn[0]))
for i in range(3500):
    posMax=6;
    valMax=y_pred_train_nn[i][6]
    for j in range(6):
        if y_pred_train_nn[i][j] > valMax:
            valMax=y_pred_train_nn[i][j];
            posMax=j;
            
    y_pred_train_nn_etiquetas.append(f(posMax));
    if i < 1050:
        posMax=6;
        valMax=y_pred_test_nn[i][6]
        for j in range(6):
            if y_pred_test_nn[i][j] > valMax:
                valMax=y_pred_test_nn[i][j];
                posMax=j;
        y_pred_test_nn_etiquetas.append(f(posMax));   
        
y_pred_train_nn_etiquetas=np.asarray(y_pred_train_nn_etiquetas);
y_pred_test_nn_etiquetas=np.asarray(y_pred_test_nn_etiquetas);

print(y_pred_train_nn_etiquetas.shape,y_pred_train_nn_etiquetas,Y_train);
print(y_pred_test_nn_etiquetas.shape,y_pred_test_nn_etiquetas, Y_test);

(3500,) [51 51 51 ... 51 51 51] [28 10 28 ... 51 75 51]
(1050,) [28 51 51 ... 51 51 51] [28 51 51 ... 51 51 10]


In [107]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_nn_etiquetas))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_nn_etiquetas))

ERROR MSE de training:  350.3468571428571
ERROR MSE de test:  357.0419047619048


Agregando $ metrics=["acc"]$ a los parámetros de compile:

In [116]:
import time
start_time =time.time()
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(y_onehot_train.shape[1], activation="softmax"))
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["acc"])
model.fit(X_train_scaled, y_onehot_train, epochs=40, batch_size=128, verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/40
3500/3500 [==============================] - 1s 395us/step - loss: 1.4930 - acc: 0.4477
Epoch 2/40
3500/3500 [==============================] - 0s 40us/step - loss: 1.2327 - acc: 0.4709
Epoch 3/40
3500/3500 [==============================] - 0s 38us/step - loss: 1.2038 - acc: 0.4734
Epoch 4/40
3500/3500 [==============================] - 0s 41us/step - loss: 1.1972 - acc: 0.4620
Epoch 5/40
3500/3500 [==============================] - 0s 43us/step - loss: 1.1888 - acc: 0.4706
Epoch 6/40
3500/3500 [==============================] - 0s 37us/step - loss: 1.1851 - acc: 0.4817
Epoch 7/40
3500/3500 [==============================] - 0s 37us/step - loss: 1.1811 - acc: 0.4789
Epoch 8/40
3500/3500 [==============================] - 0s 37us/step - loss: 1.1799 - acc: 0.4726
Epoch 9/40
3500/3500 [==============================] - 0s 42us/step - loss: 1.1759 - acc: 0.4851
Epoch 10/40
3500/3500 [==============================] - 0s 41us/step - loss: 1.1782 - acc: 0.4720
Epoch 11/40
3500/3

In [117]:
y_pred_train_nn=model.predict(X_train_scaled);
y_pred_test_nn=model.predict(X_test_scaled);

In [118]:
def f(x):
    if x==0:
        return 1;
    elif x==1:
        return 5;
    elif x==2:
        return 10;
    elif x==3:
        return 16;
    elif x==4:
        return 28;
    elif x==5:
        return 51;
    elif x==6:
        return 75;


y_pred_train_nn_etiquetas=[];
y_pred_test_nn_etiquetas=[];
#print(len(y_pred_train_nn[0]))
for i in range(3500):
    posMax=6;
    valMax=y_pred_train_nn[i][6]
    for j in range(6):
        if y_pred_train_nn[i][j] > valMax:
            valMax=y_pred_train_nn[i][j];
            posMax=j;
            
    y_pred_train_nn_etiquetas.append(f(posMax));
    if i < 1050:
        posMax=6;
        valMax=y_pred_test_nn[i][6]
        for j in range(6):
            if y_pred_test_nn[i][j] > valMax:
                valMax=y_pred_test_nn[i][j];
                posMax=j;
        y_pred_test_nn_etiquetas.append(f(posMax));   
        
y_pred_train_nn_etiquetas=np.asarray(y_pred_train_nn_etiquetas);
y_pred_test_nn_etiquetas=np.asarray(y_pred_test_nn_etiquetas);

print(y_pred_train_nn_etiquetas.shape,y_pred_train_nn_etiquetas,Y_train);
print(y_pred_test_nn_etiquetas.shape,y_pred_test_nn_etiquetas, Y_test);

(3500,) [28 51 51 ... 51 51 51] [28 10 28 ... 51 75 51]
(1050,) [28 51 51 ... 51 51 51] [28 51 51 ... 51 51 10]


In [119]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_nn_etiquetas))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_nn_etiquetas))

ERROR MSE de training:  335.85685714285717
ERROR MSE de test:  350.51428571428573


Agregando $ metrics=["mae"]$ a los parámetros de compile:

In [121]:
import time
start_time =time.time()
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(y_onehot_train.shape[1], activation="softmax"))
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["mae"])
model.fit(X_train_scaled, y_onehot_train, epochs=40, batch_size=128, verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/40
3500/3500 [==============================] - 1s 413us/step - loss: 1.4723 - mean_absolute_error: 0.2087
Epoch 2/40
3500/3500 [==============================] - 0s 49us/step - loss: 1.2192 - mean_absolute_error: 0.1823
Epoch 3/40
3500/3500 [==============================] - 0s 49us/step - loss: 1.1987 - mean_absolute_error: 0.1804
Epoch 4/40
3500/3500 [==============================] - 0s 36us/step - loss: 1.1883 - mean_absolute_error: 0.1809
Epoch 5/40
3500/3500 [==============================] - 0s 49us/step - loss: 1.1849 - mean_absolute_error: 0.1805
Epoch 6/40
3500/3500 [==============================] - 0s 45us/step - loss: 1.1804 - mean_absolute_error: 0.1800
Epoch 7/40
3500/3500 [==============================] - 0s 40us/step - loss: 1.1794 - mean_absolute_error: 0.1800
Epoch 8/40
3500/3500 [==============================] - 0s 49us/step - loss: 1.1789 - mean_absolute_error: 0.1793
Epoch 9/40
3500/3500 [==============================] - 0s 49us/step - loss: 1.1762 - m

In [122]:
y_pred_train_nn=model.predict(X_train_scaled);
y_pred_test_nn=model.predict(X_test_scaled);

In [123]:
def f(x):
    if x==0:
        return 1;
    elif x==1:
        return 5;
    elif x==2:
        return 10;
    elif x==3:
        return 16;
    elif x==4:
        return 28;
    elif x==5:
        return 51;
    elif x==6:
        return 75;


y_pred_train_nn_etiquetas=[];
y_pred_test_nn_etiquetas=[];
#print(len(y_pred_train_nn[0]))
for i in range(3500):
    posMax=6;
    valMax=y_pred_train_nn[i][6]
    for j in range(6):
        if y_pred_train_nn[i][j] > valMax:
            valMax=y_pred_train_nn[i][j];
            posMax=j;
            
    y_pred_train_nn_etiquetas.append(f(posMax));
    if i < 1050:
        posMax=6;
        valMax=y_pred_test_nn[i][6]
        for j in range(6):
            if y_pred_test_nn[i][j] > valMax:
                valMax=y_pred_test_nn[i][j];
                posMax=j;
        y_pred_test_nn_etiquetas.append(f(posMax));   
        
y_pred_train_nn_etiquetas=np.asarray(y_pred_train_nn_etiquetas);
y_pred_test_nn_etiquetas=np.asarray(y_pred_test_nn_etiquetas);

print(y_pred_train_nn_etiquetas.shape,y_pred_train_nn_etiquetas,Y_train);
print(y_pred_test_nn_etiquetas.shape,y_pred_test_nn_etiquetas, Y_test);

(3500,) [51 51 51 ... 51 51 51] [28 10 28 ... 51 75 51]
(1050,) [51 51 51 ... 51 51 51] [28 51 51 ... 51 51 10]


In [124]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_nn_etiquetas))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_nn_etiquetas))

ERROR MSE de training:  360.3222857142857
ERROR MSE de test:  358.0933333333333


Veamos ahora con un árbol de decisión de clasificación.

In [125]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=10,criterion='gini',splitter='best');
model.fit(X_train_scaled,Y_train);

In [126]:
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);

In [128]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  281.7934285714286
ERROR MSE de test:  393.1457142857143


Variaremos la profundidad máxima.

In [131]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=20,criterion='gini',splitter='best');
model.fit(X_train_scaled,Y_train);

In [132]:
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);

In [133]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  85.15342857142858
ERROR MSE de test:  493.4428571428571


In [134]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=30,criterion='gini',splitter='best');
model.fit(X_train_scaled,Y_train);

In [135]:
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);

In [136]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  19.553142857142856
ERROR MSE de test:  523.472380952381


In [140]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=8,criterion='gini',splitter='best');
model.fit(X_train_scaled,Y_train);

In [141]:
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);

In [142]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  320.0185714285714
ERROR MSE de test:  378.36380952380955


In [146]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=5,criterion='gini',splitter='best');
model.fit(X_train_scaled,Y_train);

In [147]:
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);

In [148]:
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  333.62857142857143
ERROR MSE de test:  365.9104761904762


Cambiando el criterio por $entropy$, y variando la profundidad.  

In [167]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=5,criterion='entropy',splitter='best');
model.fit(X_train_scaled,Y_train);
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  352.15171428571426
ERROR MSE de test:  372.05619047619047


In [175]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=4,criterion='entropy',splitter='best');
model.fit(X_train_scaled,Y_train);
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  358.82428571428574
ERROR MSE de test:  364.1857142857143


In [176]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=8,criterion='entropy',splitter='best');
model.fit(X_train_scaled,Y_train);
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  311.4611428571429
ERROR MSE de test:  392.13428571428574


In [177]:
from sklearn.tree import DecisionTreeClassifier as Tree;
model = Tree() ;
model.set_params(max_depth=10,criterion='entropy',splitter='best');
model.fit(X_train_scaled,Y_train);
y_pred_train_tree=model.predict(X_train_scaled);
y_pred_test_tree=model.predict(X_test_scaled);
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_tree))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_tree))

ERROR MSE de training:  268.93514285714286
ERROR MSE de test:  410.0933333333333


Ahora con SVM para clasificación.  

In [180]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsRestClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=1,kernel='rbf')
model = OneVsRestClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 2.774691343307495 seconds ---
ERROR MSE de training:  387.69885714285715
ERROR MSE de test:  401.35238095238094


In [194]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsOneClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=1,kernel='rbf')
model = OneVsOneClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 2.6108217239379883 seconds ---
ERROR MSE de training:  354.00714285714287
ERROR MSE de test:  371.0390476190476


In [181]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsRestClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=0.1,kernel='rbf')
model = OneVsRestClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 2.091035842895508 seconds ---
ERROR MSE de training:  398.83257142857144
ERROR MSE de test:  390.5628571428571


In [195]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsOneClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=0.1,kernel='rbf')
model = OneVsOneClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 2.4388513565063477 seconds ---
ERROR MSE de training:  368.6614285714286
ERROR MSE de test:  368.82666666666665


In [183]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsRestClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=0.01,kernel='rbf')
model = OneVsRestClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 1.8593816757202148 seconds ---
ERROR MSE de training:  411.2168571428571
ERROR MSE de test:  395.3504761904762


In [196]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsOneClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=0.01,kernel='rbf')
model = OneVsOneClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 2.1718966960906982 seconds ---
ERROR MSE de training:  368.6614285714286
ERROR MSE de test:  368.82666666666665


In [184]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsRestClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=10,kernel='rbf')
model = OneVsRestClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 8.422327280044556 seconds ---
ERROR MSE de training:  366.078
ERROR MSE de test:  387.1647619047619


In [197]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsOneClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=10,kernel='rbf')
model = OneVsOneClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 5.032118797302246 seconds ---
ERROR MSE de training:  348.2628571428571
ERROR MSE de test:  363.5904761904762


In [187]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsRestClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=1,kernel='poly')
model = OneVsRestClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 2.3757967948913574 seconds ---
ERROR MSE de training:  388.88514285714285
ERROR MSE de test:  435.70190476190476


In [192]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsOneClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=1,kernel='poly')
model = OneVsOneClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 1.7274479866027832 seconds ---
ERROR MSE de training:  368.6614285714286
ERROR MSE de test:  368.82666666666665


In [188]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsRestClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=0.1,kernel='poly')
model = OneVsRestClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 1.6004409790039062 seconds ---
ERROR MSE de training:  394.66857142857145
ERROR MSE de test:  410.4895238095238


In [191]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsOneClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=0.1,kernel='poly')
model = OneVsOneClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 1.4810547828674316 seconds ---
ERROR MSE de training:  368.6614285714286
ERROR MSE de test:  368.82666666666665


In [189]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsRestClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=10,kernel='poly')
model = OneVsRestClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 6.295992136001587 seconds ---
ERROR MSE de training:  390.23657142857144
ERROR MSE de test:  430.807619047619


In [190]:
from sklearn.svm import SVC as SVM 
from sklearn.multiclass import OneVsOneClassifier
import time
start_time =time.time()
model= SVM()
model.set_params(C=10,kernel='poly')
model = OneVsOneClassifier(model)
model.fit(X_train_scaled,Y_train)
y_pred_train_svm=model.predict(X_train_scaled);
y_pred_test_svm=model.predict(X_test_scaled);
print("--- %s seconds ---" % (time.time() - start_time))
print("ERROR MSE de training: ", MSE(Y_train,y_pred_train_svm))
print("ERROR MSE de test: ", MSE(Y_test,y_pred_test_svm))

--- 2.953841209411621 seconds ---
ERROR MSE de training:  368.6614285714286
ERROR MSE de test:  368.82666666666665


Se experimenta con múltiples modelos variando sus parámetros, sin embargo, está muy lejos de llegar al error pedido. Esto lo atribuimos al hecho de que las representación usada y los atributos elegidos no son los óptimos para realizar buenas predicciones. Quizas se podria mejorar la representación, tomando otras clases como representativas, o directamente hacen falta más ejemplos. Otro motivo del alto error puede ser el que el hecho de tratar el problema como uno de clasificación, hace que no se permitan edades intermedias, de ahí que no se reduzcan tanto los errores MSE.

EL mínimo error de test se obtuvo con el siguiente modelo y parámetros:

In [ ]:
import time
start_time =time.time()
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(y_onehot_train.shape[1], activation="softmax"))
model.compile(optimizer="Adam", loss="categorical_crossentropy")
model.fit(X_train_scaled, y_onehot_train, epochs=15, batch_size=128, verbose=1)

print("--- %s seconds ---" % (time.time() - start_time))

Obteniéndose:  
    

ERROR MSE de training:  341.114  
ERROR MSE de test:  348.1485714285714  

#### Ayuda:

Para problemas de clasificación de múltiples clases, la red neuronal de *keras* necesita una represnetacion *one hot vector* similar a lo comentado en la sección 2, por lo que será necesario transformar/codificar las edades a etiquetas categóricas, donde cada columna del vector representará una categoría. Por ejemplo, si existen tres categorías (perro, gato, ratón), la categoría perro puede ser codificada como [1,0,0], y la categoría ratón puede ser codificada como [0,0,1]. Para ésto la librería *keras* nos ayuda:

<div class="alert alert-warning"> Recuerde que si trabaja el problema como clasificación deberá invertir la transformación de codificación de las edades a clases, para así poder evaluar el MSE </div>

*Recuerde que:* Si encuentra que la métrica evaluadora le perjudica puede acudir a otras para entender el qué está pasando con su modelo.

<a id="refs"></a>
## Referencias
[1] Keras: Deep Learning library for Theano and TensorFlow. https://keras.io/  
[2] http://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html  
[3] https://en.wikipedia.org/wiki/Kernel_method  
[4] http://scikit-learn.org/stable/modules/multiclass.html  
[5] Tsoumakas, G., & Katakis, I. (2007). *Multi-label classification: An overview*. International Journal of Data Warehousing and Mining (IJDWM), 3(3), 1-13.  
[6] https://medium.com/open-machine-learning-course/open-machine-learning-course-topic-6-feature-engineering-and-feature-selection-8b94f870706a  
[7] Bishop, C. M. (2006). *Pattern recognition and machine learning (information science and statistics)* springer-verlag new york. Inc. Secaucus, NJ, USA.